In [1]:
import os, sys, time, random
import pandas as pd
import numpy as np
from os.path import join
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import preprocessing
from sklearn.cross_validation import StratifiedKFold

sys.path.append('/home/ymm/kaggle/xgboost_hyperopt')
import utils.bosch_functions as bosch_functions
from utils.wrapped_xgboost import xgboost_classifier
from utils.validation_tools import score_MCC, MCC, create_validation_index
from utils.models import CombinedModel
from utils.data_munge import remove_single_value_columns
from utils.feature_engineering import NumericalFeatureEngineering, getRelativeTimeColumns, BasicDate_FeatureEngineering
from utils.feature_engineering import getTimeChangeColumns, getTimeSteps, build_IndexFeatures, build_sortedData_indexDiff
from utils.feature_engineering import BasicCat_FeatureEngineering, encode_categorical_by_dep_var


data_path = '/home/ymm/bosch/'

train_num_file   = 'train_numeric.csv'
train_cat_file   = 'train_categorical.csv'
train_date_file  = 'train_date.csv'
test_num_file    = 'test_numeric.csv'
test_cat_file    = 'test_categorical.csv'
test_date_file   = 'test_date.csv'

sample_submission_file   = 'sample_submission.csv'

start_time_column_name = 'L0_S0_D1'
id_column_name = 'Id'
dep_var_name = 'Response'

In [2]:
tot_row_num = 1183747
num_rows = 50000
skip = sorted(random.sample(xrange(1,tot_row_num + 1),tot_row_num - num_rows))

In [3]:
#'''
start_time = time.time()
## randomly select certain rows
train_num = pd.read_csv(join(data_path, train_num_file),    index_col='Id', skiprows=skip, nrows=num_rows)
train_dat = pd.read_csv(join(data_path, train_date_file),   index_col='Id', skiprows=skip, nrows=num_rows)
train_cat = pd.read_csv(join(data_path, train_cat_file),    index_col='Id', skiprows=skip, nrows=num_rows)

test_num = pd.read_csv(join(data_path, test_num_file),      index_col='Id', nrows=num_rows)
test_dat = pd.read_csv(join(data_path, test_date_file),     index_col='Id', nrows=num_rows)
test_cat = pd.read_csv(join(data_path, test_cat_file),      index_col='Id', nrows=num_rows)

print 'finish loading date using {} seconds'.format(round(time.time() - start_time, 0))
#'''

finish loading date using 90.0 seconds


In [4]:
remove_single_value_columns(train_num, 'Response', test=test_num)
remove_single_value_columns(train_dat, test=test_dat)
remove_single_value_columns(train_cat, test=test_cat)

raw train data dimension:  (50000, 969)
raw test data dimension:  (50000, 968)
processed train data dimension:  (50000, 969)
processed test data dimension:  (50000, 968)
raw train data dimension:  (50000, 1156)
raw test data dimension:  (50000, 1156)
processed train data dimension:  (50000, 1146)
processed test data dimension:  (50000, 1146)
raw train data dimension:  (50000, 2140)
raw test data dimension:  (50000, 2140)
processed train data dimension:  (50000, 1311)
processed test data dimension:  (50000, 1311)


In [5]:
dat_columns = train_dat.columns.tolist()
num_columns = train_num.columns.tolist()
num_columns.remove(dep_var_name)

def build_column_dict(columns):
    station_dict = {}
    line_dict = {}
    for col in columns:
        stationList = col.split('_')[0:2]
        stationKey = ('_').join(stationList)
        lineKey = col.split('_')[0]
        
        if lineKey not in line_dict:
            line_dict[lineKey] = [col]
        else:
            line_dict[lineKey].append(col)
                    
        if stationKey not in station_dict:
            station_dict[stationKey] = [col]
        else:
            station_dict[stationKey].append(col)
            
    return station_dict, line_dict


def build_station_features(df, col_dict, prefix='dat'):
    features = pd.DataFrame()
    for key, value in col_dict.items():
        features['{}_{}_{}'.format(prefix, key, 'mean')] = df[value].mean(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'max')] = df[value].max(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'min')] = df[value].min(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'var')] = df[value].var(axis=1)
    return features


def build_station_index_features(train, test = None):
    selected_columns = []
    for col in train.columns:
        if 'mean' in col or 'var' in col:
            selected_columns.append(col)
            
    if test is not None:
        train_test = pd.concat([train[selected_columns], test[selected_columns]], axis=0)
    else:
        train_test = train[selected_columns]
        
    train_test['index'] = train_test.index
    new_fea = pd.DataFrame()
    ## function to build index based on the given columns
    build_sortedData_indexDiff(train_test, new_fea, selected_columns)
    
    return new_fea


dat_col_dict, dat_line_dict = build_column_dict(dat_columns)
num_col_dict, num_line_dict = build_column_dict(num_columns)

dat_col_dict.update(dat_line_dict)
num_col_dict.update(num_line_dict)

In [6]:
start_time = time.time()

train_dat_stations = build_station_features(train_dat, dat_col_dict, 'dat')
test_dat_stations = build_station_features(test_dat, dat_col_dict, 'dat')

train_num_stations = build_station_features(train_num, num_col_dict, 'num')
test_num_stations = build_station_features(test_num, num_col_dict, 'num')

num_station_index = build_station_index_features(train_num_stations, test_num_stations)
dat_station_index = build_station_index_features(train_dat_stations, test_dat_stations)

print 'finish feature engineering date station using {} minutes'.format(round((time.time() - start_time)/60, 2))

finish feature engineering date station using 0.95 minutes


In [7]:
combined_train_station_num = pd.concat([train_num_stations, num_station_index.ix[train_num_stations.index]], axis=1)
combined_train_station_dat = pd.concat([train_dat_stations, dat_station_index.ix[train_dat_stations.index]], axis=1)

combined_test_station_num = pd.concat([test_num_stations, num_station_index.ix[test_num_stations.index]], axis=1)
combined_test_station_dat = pd.concat([test_dat_stations, dat_station_index.ix[test_num_stations.index]], axis=1)

In [8]:
print combined_train_station_num.isnull().sum().sum(), combined_train_station_dat.isnull().sum().sum() 

7909914 7909914


In [9]:
print combined_test_station_num.isnull().sum().sum(), combined_test_station_dat.isnull().sum().sum() 

7908313 7908313


In [10]:
remove_single_value_columns(combined_train_station_num, test=combined_test_station_num)
remove_single_value_columns(combined_train_station_dat, test=combined_test_station_dat)

raw train data dimension:  (50000, 432)
raw test data dimension:  (50000, 432)
processed train data dimension:  (50000, 431)
processed test data dimension:  (50000, 431)
raw train data dimension:  (50000, 432)
raw test data dimension:  (50000, 432)
processed train data dimension:  (50000, 431)
processed test data dimension:  (50000, 431)


In [11]:
station_fillna_value = 9999999
combined_train_station_num.fillna(station_fillna_value, inplace=True)
combined_test_station_num.fillna(station_fillna_value, inplace=True)
combined_train_station_dat.fillna(station_fillna_value, inplace=True)
combined_test_station_dat.fillna(station_fillna_value, inplace=True)

In [12]:
combined_train_station_num.head()

,num_L2_S28_mean,num_L2_S28_max,num_L2_S28_min,num_L2_S28_var,num_L3_S31_mean,num_L3_S31_max,num_L3_S31_min,num_L3_S31_var,num_L2_S26_mean,num_L2_S26_max,...,num_L1_S25_var_index_diff_0,num_L1_S25_var_index_diff_1,num_L1_S24_mean_index_diff_0,num_L1_S24_mean_index_diff_1,num_L1_S24_var_index_diff_0,num_L1_S24_var_index_diff_1,num_L3_S39_mean_index_diff_0,num_L3_S39_mean_index_diff_1,num_L3_S39_var_index_diff_0,num_L3_S39_var_index_diff_1
Id,,,,,,,,,,,,,,,,,,,,,
28,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9.999999e+06,9999999.000,...,3,-1,3,-1,3,-1,3,-1,3,-1
140,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9.999999e+06,9999999.000,...,1,-1,1,-1,1,-1,31,-7,31,-7
175,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9.999999e+06,9999999.000,...,3,-3,2,-3,2,-3,2,-3,2,-3
245,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9.999999e+06,9999999.000,...,2,-6,2,-6,2,-6,2,-6,2,-6
339,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,5.971429e-02,0.299,...,1,-1,-2069228,-67667,-308753,-274481,1,-1,1,-1


In [13]:
print combined_train_station_num.isnull().sum().sum(), combined_train_station_dat.isnull().sum().sum()

0 0


#### build categorical features

In [14]:
'''
def BasicCat_FeatureEngineering(train_cat):
    ## feature engineering on the date features
    encoder = preprocessing.LabelEncoder()
    column_names = train_cat.columns.tolist()
    column_names.append('NaN')
    encoder.fit(column_names)
    dat_new_fea = pd.DataFrame()
    dat_new_fea['cat_sum'] = train_cat.sum(axis=1)
    dat_new_fea['cat_mean'] = train_cat.mean(axis=1)
    dat_new_fea['cat_nan_count'] = train_cat.isnull().sum(axis=1)
    dat_new_fea['cat_max'] = train_cat.max(axis=1)
    dat_new_fea['cat_min'] = train_cat.min(axis=1)
    dat_new_fea['cat_max_min_diff'] = dat_new_fea['cat_max'] - dat_new_fea['cat_min']
    dat_new_fea['cat_max_min_ratio'] = dat_new_fea['cat_min'] / dat_new_fea['cat_max']

    dat_new_fea['cat_idxmax'] = train_cat.idxmax(axis=1)
    dat_new_fea['cat_idxmax'].fillna('NaN', inplace=True)
    dat_new_fea['cat_idxmax'] = encoder.transform(dat_new_fea['cat_idxmax'])
    dat_new_fea['cat_idxmin'] = train_cat.idxmin(axis=1)
    dat_new_fea['cat_idxmin'].fillna('NaN', inplace=True)
    dat_new_fea['cat_idxmin'] = encoder.transform(dat_new_fea['cat_idxmin'])
    return dat_new_fea



def encode_categorical_by_dep_var(train, test, dep_var_column='Response'):
    for col_name in train.columns:
        if col_name == dep_var_column:
            continue
        
        train[col_name] = train[col_name].astype(str)
        test[col_name] = test[col_name].astype(str)
        dep_var_mean = train[[col_name, dep_var_column]].groupby(col_name).mean()
    
        dep_var_dict = {}
        for level in dep_var_mean.index.tolist():
            dep_var_dict[level] = dep_var_mean.ix[level, dep_var_column]
    
        train[col_name] = train[col_name].replace(dep_var_dict)  
        test[col_name] = test[col_name].replace(dep_var_dict)
'''

"\ndef BasicCat_FeatureEngineering(train_cat):\n    ## feature engineering on the date features\n    encoder = preprocessing.LabelEncoder()\n    column_names = train_cat.columns.tolist()\n    column_names.append('NaN')\n    encoder.fit(column_names)\n    dat_new_fea = pd.DataFrame()\n    dat_new_fea['cat_sum'] = train_cat.sum(axis=1)\n    dat_new_fea['cat_mean'] = train_cat.mean(axis=1)\n    dat_new_fea['cat_nan_count'] = train_cat.isnull().sum(axis=1)\n    dat_new_fea['cat_max'] = train_cat.max(axis=1)\n    dat_new_fea['cat_min'] = train_cat.min(axis=1)\n    dat_new_fea['cat_max_min_diff'] = dat_new_fea['cat_max'] - dat_new_fea['cat_min']\n    dat_new_fea['cat_max_min_ratio'] = dat_new_fea['cat_min'] / dat_new_fea['cat_max']\n\n    dat_new_fea['cat_idxmax'] = train_cat.idxmax(axis=1)\n    dat_new_fea['cat_idxmax'].fillna('NaN', inplace=True)\n    dat_new_fea['cat_idxmax'] = encoder.transform(dat_new_fea['cat_idxmax'])\n    dat_new_fea['cat_idxmin'] = train_cat.idxmin(axis=1)\n    dat_

In [15]:
start_time = time.time()

train_cat['Response'] = train_num['Response']
encode_categorical_by_dep_var(train_cat, test_cat, dep_var_column='Response', fill_missing=True)
train_cat.drop('Response', axis=1, inplace=True)

print 'finish encoding categorical features using {} seconds'.format(round(time.time() - start_time, 0))

train_cat_Basics = BasicCat_FeatureEngineering(train_cat)
test_cat_Basics  = BasicCat_FeatureEngineering(test_cat)

print 'finish generating all categorical features using {} seconds'.format(round(time.time() - start_time, 0))


finish encoding categorical features using 9.0 seconds
finish generating all categorical features using 24.0 seconds


In [16]:
print train_cat.shape, test_cat.shape
train_cat.head()

(50000, 1311) (50000, 1311)


,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,L0_S2_F43,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
Id,,,,,,,,,,,,,,,,,,,,,
28,5.42,5.42,5.42,5.42,5.421,5.421,5.421,5.421,5.421,5.421,...,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42
140,5.42,5.42,5.42,5.42,5.421,5.421,5.421,5.421,5.421,5.421,...,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42
175,5.42,5.42,5.42,5.42,5.421,5.421,5.421,5.421,5.421,5.421,...,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42
245,5.42,5.42,5.42,5.42,5.421,5.421,5.421,5.421,5.421,5.421,...,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42
339,5.42,5.42,5.42,5.42,5.421,5.421,5.421,5.421,5.421,5.421,...,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42,5.42


In [17]:
start_time = time.time()
combined_train_cat = pd.concat([train_cat, train_cat_Basics], axis=1)
combined_test_cat  = pd.concat([test_cat, test_cat_Basics], axis=1)                                                                                                                                                 
print 'finish feature engineering date using {} seconds'.format(round((time.time() - start_time), 2))

finish feature engineering date using 0.52 seconds


In [18]:
print combined_train_cat.isnull().sum(axis=0).sum(), combined_test_cat.isnull().sum(axis=0).sum()

0 0


In [19]:
combined_test_cat.head()

,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,L0_S2_F43,...,L3_S49_F4240,cat_sum,cat_mean,cat_nan_count,cat_max,cat_min,cat_max_min_diff,cat_max_min_ratio,cat_idxmax,cat_idxmin
Id,,,,,,,,,,,,,,,,,,,,,
1,5.42,5.42,5.42,5.42,5.421,5.421,5.421,5.421,5.421,5.421,...,5.42,7227.472,5.512946,0,8.555,4.867,3.688,0.568907,388,1234
2,5.42,5.42,5.42,5.42,5.421,5.421,5.421,5.421,5.421,5.421,...,5.42,6929.384,5.285571,0,5.574,1.883,3.691,0.337818,1091,529
3,5.42,5.42,5.42,5.42,5.421,5.421,5.421,5.421,5.421,5.421,...,5.42,7281.905,5.554466,0,19.685,4.867,14.818,0.247244,389,1234
5,5.42,5.42,5.42,5.42,5.421,5.421,5.421,5.421,5.421,5.421,...,5.42,7089.684,5.407844,0,6.440,4.267,2.173,0.662578,1038,1074
8,5.42,5.42,5.42,5.42,5.421,5.421,5.421,5.421,5.421,5.421,...,5.42,7338.145,5.597365,0,19.685,4.867,14.818,0.247244,389,1234


In [20]:
combined_test_cat.isnull().sum().sum()

0

In [21]:
start_time = time.time()

#### numerical feature engineering work
train_num_Basics = NumericalFeatureEngineering(train_num)
test_num_Basics = NumericalFeatureEngineering(test_num)

missing_value= -1.5
train_num_Basics.fillna(missing_value, inplace=True)
test_num_Basics.fillna(missing_value, inplace=True)

train_num.fillna(missing_value, inplace=True)
test_num.fillna(missing_value, inplace=True)

combined_train_num = pd.concat([train_num, train_num_Basics], axis=1)
combined_test_num  = pd.concat([test_num, test_num_Basics], axis=1)
print 'finish feature engineering numercical using {} seconds'.format(round((time.time() - start_time), 2))
print 'combined train numerical feature shape: {}, combined test numerical features shape: {}'.format(combined_train_num.shape, combined_test_num.shape)

finish feature engineering numercical using 23.32 seconds
combined train numerical feature shape: (50000, 978), combined test numerical features shape: (50000, 977)


In [22]:
print combined_train_num.isnull().sum().sum(), combined_test_num.isnull().sum().sum()

0 0


### section of date features

In [23]:

## basic features from tmp_train_dat
train_dat_Basics = BasicDate_FeatureEngineering(train_dat)
test_dat_Basics  = BasicDate_FeatureEngineering(test_dat)

In [24]:
## normalized date columns
train_dat_Norm = train_dat.apply(getRelativeTimeColumns, axis=1)
test_dat_Norm  = test_dat.apply(getRelativeTimeColumns, axis=1)
## remove single-valued columns
remove_single_value_columns(train_dat_Norm, test=test_dat_Norm)

raw train data dimension:  (50000, 1146)
raw test data dimension:  (50000, 1146)
processed train data dimension:  (50000, 960)
processed test data dimension:  (50000, 960)


In [ ]:
encoder = preprocessing.LabelEncoder()
column_names = train_dat.columns.tolist()
column_names.append('NaN')
encoder.fit(column_names)

LabelEncoder()

In [ ]:
## TimeDiff features
train_dat_TimeDiff = train_dat.apply(getTimeChangeColumns, axis=1)
test_dat_TimeDiff  = test_dat.apply(getTimeChangeColumns, axis=1)
TimeDiff_ColumnNames = ['time_diff_start_col', 'time_diff_end_col', 'time_diff_value',
                        'time_ratio_value', 'first_time_value', 'last_time_value', 'first_date_value']

train_dat_TimeDiff.columns = TimeDiff_ColumnNames
test_dat_TimeDiff.columns = TimeDiff_ColumnNames

for column in ['time_diff_start_col', 'time_diff_end_col']:
    train_dat_TimeDiff[column].fillna('NaN', inplace=True)
    train_dat_TimeDiff[column] = encoder.transform(train_dat_TimeDiff[column])
    
    test_dat_TimeDiff[column].fillna('NaN', inplace=True)
    test_dat_TimeDiff[column] = encoder.transform(test_dat_TimeDiff[column])


In [ ]:
start_time = time.time()

## section to create timeStep features
unique_value_counts = 6
timeStep_columnNames = []
column_name_columns = []
for i in xrange(unique_value_counts):
    timeStep_columnNames.extend(['time_diff_step_{}'.format(i), 'column_counts_step_{}'.format(i),
                                 'time_cost_step_{}'.format(i), 'first_column_step_{}'.format(i)])
    column_name_columns.append('first_column_step_{}'.format(i))

train_dat_TimeStep = train_dat_Norm.apply(getTimeSteps, axis=1)
test_dat_TimeStep  = test_dat_Norm.apply(getTimeSteps, axis=1)
train_dat_TimeStep.columns = timeStep_columnNames
test_dat_TimeStep.columns  = timeStep_columnNames

for column in column_name_columns:
    train_dat_TimeStep[column].fillna('NaN', inplace=True)
    test_dat_TimeStep[column].fillna('NaN', inplace=True)
    train_dat_TimeStep[column] = encoder.transform(train_dat_TimeStep[column])
    test_dat_TimeStep[column] = encoder.transform(test_dat_TimeStep[column])


print 'finish generating TimeStep features using {} minutes'.format(round((time.time() - start_time)/60, 0))


In [ ]:
#print train_dat_Norm.isnull().sum().sum(), train_dat_Basics.isnull().sum().sum(), train_dat_TimeStep.isnull().sum().sum(), train_dat_TimeDiff.isnull().sum().sum()

##### fill up missing values

In [ ]:
'''
tmp_train_dat = train_dat_TimeDiff.copy()
tmp_test_dat = test_dat_TimeDiff.copy()

if 'start_time' in train_dat_Basics:
    tmp_train_dat['start_time'] = train_dat_Basics['start_time']
    tmp_test_dat['start_time']  = test_dat_Basics['start_time']
'''


In [ ]:
start_time = time.time()
combined_train_dat = pd.concat([train_dat_Norm, train_dat_Basics, train_dat_TimeDiff, train_dat_TimeStep], axis=1)
combined_test_dat  = pd.concat([test_dat_Norm, test_dat_Basics, test_dat_TimeDiff, test_dat_TimeStep], axis=1)                                                                                                                                                 
print 'finish feature engineering date using {} minutes'.format(round((time.time() - start_time)/60, 2))

In [ ]:
start_time = time.time()
train_test_datIndex_features = build_IndexFeatures(combined_train_dat, combined_test_dat)
print 'finish feature engineering date index using {} minutes'.format(round((time.time() - start_time)/60, 2))

#### index columns fill up missing

In [ ]:
train_test_datIndex_features.fillna(-1., inplace=True)

#### after filling up missing, re-create the date columns

In [ ]:
print combined_train_dat.shape, combined_test_dat.shape
combined_train_dat.head()

In [ ]:
train_dat_Norm.fillna(-1., inplace=True)
test_dat_Norm.fillna(-1., inplace=True)

train_dat_Basics.fillna(-1., inplace=True)
test_dat_Basics.fillna(-1., inplace=True)

train_dat_TimeStep.fillna(0, inplace=True)
test_dat_TimeStep.fillna(0, inplace=True)

train_dat_TimeDiff.fillna(0, inplace=True)
test_dat_TimeDiff.fillna(0, inplace=True)

In [ ]:
combined_train_dat = pd.concat([train_dat_Norm, train_dat_Basics, train_dat_TimeDiff, train_dat_TimeStep], axis=1)
combined_test_dat  = pd.concat([test_dat_Norm, test_dat_Basics, test_dat_TimeDiff, test_dat_TimeStep], axis=1)  

In [ ]:
print combined_test_dat.isnull().sum().sum()

In [ ]:
#train_test_datIndex_features['index_ratio'].min()
#train_test_datIndex_features.isnull().sum().sum()

#### load the feature importances from xgboost models

In [ ]:
xgb_feature_imp = pd.read_csv('/home/ymm/full_data_xgb_feature_importance.csv', index_col='feature')

In [ ]:
xgb_feature_imp.head()

In [ ]:
## sort by the norm_fscore_sum
sorted_combined_imp = xgb_feature_imp.sort_values(by=['norm_fscore_sum'], ascending=False)
#imp_feature = sorted_combined_imp.index[sorted_combined_imp['norm_fscore_sum'] >= 0.005].tolist()
imp_feature = sorted_combined_imp.index.tolist()

In [ ]:
def select_important_features(df, imp_feature, test_df = None, dep_var_name = 'Response'):
    imp_col_names = [col for col in df.columns if col in imp_feature]
    print 'total {} columns in original DataFrame, select {} columns'.format(df.shape[1], len(imp_col_names))
    train_col_names = imp_col_names[:]
    test_col_names = imp_col_names[:]
    if dep_var_name in df.columns:    
        train_col_names.append(dep_var_name)
    if test_df is None:
        return df[train_col_names]
    else:
        return df[train_col_names], test_df[test_col_names]


In [ ]:
combined_train_num, combined_test_num  = select_important_features(combined_train_num, imp_feature, combined_test_num)
combined_train_dat, combined_test_dat  = select_important_features(combined_train_dat, imp_feature, combined_test_dat)
train_test_datIndex_features = select_important_features(train_test_datIndex_features, imp_feature)

#### combine all the features together

In [ ]:
## combined data with station features
#combined_train = pd.concat([train_dat_stations, train_num_stations, combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([test_dat_stations, test_num_stations, combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [ ]:
#combined_train = pd.concat([combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [ ]:
## combined data with categorical features
#combined_train = pd.concat([combined_train_cat, combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([combined_test_cat,  combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [ ]:
## combined data with all features
combined_train = pd.concat([combined_train_station_dat, combined_train_station_num, combined_train_cat, combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
combined_test  = pd.concat([combined_test_station_dat, combined_test_station_num, combined_test_cat,  combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [ ]:
print combined_test_cat.isnull().sum().sum()

In [ ]:
print combined_train.isnull().sum().sum(), combined_test.isnull().sum().sum()

In [ ]:
print combined_test.shape
combined_test.head()

In [ ]:
print combined_train.shape
combined_train.head()

#### KFold cross-validation

In [ ]:
params = {}
params["eta"]                      = 0.0075
params["subsample"]                = 0.8
params["colsample_bytree"]         = 0.8
params["num_round"]                = 501
params["max_depth"]                = 5
params["gamma"]                    = 0
params["metrics"]                  = 'auc'
params['eval_metric']              = 'auc'
params["seed"]                     = 999
params['verbose_eval']             = 50
## whether to use weights
params['use_base_score']           = True
params['use_weights']              = True
#params['use_scale_pos_weight']     = True
params["val"]                      = False


In [ ]:

skf = StratifiedKFold(combined_train[dep_var_name], 4)

for train_index, valid_index in skf:
    valid_data = combined_train.iloc[valid_index]
    tmp_train  = combined_train.iloc[train_index]

    y = tmp_train[dep_var_name].values
    X = tmp_train.drop(dep_var_name, axis=1)

    valid_y = valid_data[dep_var_name].values
    valid_X = valid_data.drop(dep_var_name, axis=1)
    
    model = xgboost_classifier(label_name = dep_var_name, params = params, model_file='test_bosch_xgb_model')
    model.fit(tmp_train, dep_var_name)
    
    pred = model.predict(valid_X)
    print 'result from using constant fraction: \n', score_MCC(valid_y, pred)
    print '\n'
    print 'result from using flexsible threshold:', CombinedModel.mcc_eval_func(valid_y, pred)

In [ ]:
############## Section of regular validation #######################
train_index, valid_index = create_validation_index(combined_train, 0.3, dep_var_name, True)
valid_data = combined_train.ix[valid_index]
tmp_train  = combined_train.ix[train_index]

y = tmp_train[dep_var_name].values
X = tmp_train.drop(dep_var_name, axis=1)

valid_y = valid_data[dep_var_name].values
valid_X = valid_data.drop(dep_var_name, axis=1)


#### regular models

In [ ]:
#print tmp_train['index_ratio'].min(), tmp_train['index_ratio'].max()

In [ ]:
rf_params = {'random_state' : 9999, 'n_estimators' : 2000, 'max_depth' : 5, 'criterion' : 'gini', 'n_jobs' : -1}
et_params = {'random_state' : 9999, 'n_estimators' : 200, 'max_depth' : 5, 'criterion' : 'gini', 'n_jobs' : -1}
rf_clf = RandomForestClassifier(**rf_params)
rf_clf = rf_clf.fit(X, y)

et_clf = RandomForestClassifier(**et_params)
et_clf = et_clf.fit(X, y)

In [ ]:
rf_pred = rf_clf.predict_proba(valid_X)[:, 1]
et_pred = et_clf.predict_proba(valid_X)[:, 1]

print 'result from using constant fraction: \n'
print score_MCC(valid_y, rf_pred)
print score_MCC(valid_y, et_pred)
print '\n'
print 'result from using flexsible threshold:'
print CombinedModel.mcc_eval_func(valid_y, rf_pred)
print CombinedModel.mcc_eval_func(valid_y, et_pred)


In [ ]:
############## Section of regular validation #######################
train_index, valid_index = create_validation_index(combined_train, 0.3, dep_var_name, True)
valid_data = combined_train.ix[valid_index]
tmp_train  = combined_train.ix[train_index]

y = tmp_train[dep_var_name].values
X = tmp_train.drop(dep_var_name, axis=1)

valid_y = valid_data[dep_var_name].values
valid_X = valid_data.drop(dep_var_name, axis=1)

model = xgboost_classifier(label_name = dep_var_name, params = params, model_file='test_bosch_xgb_model')
model.fit(tmp_train, dep_var_name)
pred = model.predict(valid_X)

print 'result from using constant fraction: \n', score_MCC(valid_y, pred)
print '\n'
print 'result from using flexsible threshold:', CombinedModel.mcc_eval_func(valid_y, pred)